In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats


In [2]:
# ------------------------------------------------------------------------
# NOTE
# ------------------------------------------------------------------------
# Purpose: combine the 3 datasets: choices.dta, price_scales.dta, providers.dta 
# 
#
# Output: 
#     1. data_merged.h5 , key: cases, the cleaned and merged data
#         each row = one case, one consumer might have more than 1 cases
# ------------------------------------------------------------------------



In [414]:
# 0. Initialize file path -----------------------------------------------
rootpath = '/Users/jingyuanwang/Dropbox/Course/ECON/IO/NU450/NU450_HW/coding_tutorial'
datapath = rootpath + '/' + 'data'
resultpath = rootpath + '/' + 'results'

In [351]:
# I. Input data ---------------------------------------------------------
dtafiles = ['choices','price_scales', 'providers']
for filename in dtafiles:
    data = datapath + '/' + filename + '.dta'
    globals()[filename] = pd.read_stata(data)

In [352]:
# 1. clean choice dataset:  
# (1). drop duplicates
choices = choices.drop(columns = 'index')
choices = (choices.drop_duplicates()
           .sort_values( ['consumer_id','year'], ascending = True)
           .reset_index(drop = True))
# order variables

In [353]:
ids = ['consumer_id','year','insurer_id','provider_id']
varnames = ids + ['age', 'female', 'num_dependents']
choices = choices[varnames]

In [355]:
# (2). drop if consumer_id-year-provider has duplicates observations
# choices.drop_duplicates(['consumer_id','year', 'provider_id','insurer_id'], keep = False)
# This won't work, because we want to drop the entire consumer. This only drops the problematic person-year
dup = choices.groupby(ids).size().rename('dup')
problematic_consumer_id = dup[dup>1].index.get_level_values(0).unique().to_numpy()
choices = choices[~choices['consumer_id'].isin(problematic_consumer_id)]

In [357]:
# (3). check groupby consumer:
# (a) consumer's age increase by 1 every year
list_age_problematic = []
# (b) gender does not change across year
list_gender_problematic = []
# check:
for consumer, frame in choices.groupby('consumer_id'):
    if len(frame) > 1:
        # (1) age increase by 1 with year
        if (frame['age'].diff()[1:] != frame['year'].diff()[1:]).sum() > 0:
            list_age_problematic.append(consumer)
        # (2) gender should not variant
        if (frame['female'].var() != 0) :
            list_gender_problematic.append(consumer)

In [358]:
choices = choices[~choices['consumer_id'].isin(list_age_problematic)]
choices = choices[~choices['consumer_id'].isin(list_gender_problematic)]

In [360]:
# 2. clean prviders data
providers = (providers.drop(columns = 'index')
                    .drop_duplicates()
                    .sort_values( ['provider_id'], ascending = True)
                    .reset_index(drop = True))

In [361]:
# 3. clean price scales data

In [362]:
price_scales = (price_scales.drop_duplicates()
                    .sort_values( ['provider_id','insurer_id'], ascending = True)
                    .reset_index(drop = True))

In [363]:
for provider in choices['provider_id'].unique():
    print('---- provider: {}'.format(provider))
    print(price_scales[price_scales['provider_id'] == provider]['insurer_id'].unique())
    print(choices[choices['provider_id'] == provider]['insurer_id'].unique())

---- provider: 13
[ 67  78  80  88  99 107]
[1 5 0 3 4 2]
---- provider: 7
[ 67  78  80  88  99 107]
[0 1 2 4 3 5]
---- provider: 8
[ 67  78  80  88  99 107]
[5 1 0 2 4 3]
---- provider: 6
[ 67  78  80  88  99 107]
[0 1 5 3 2 4]
---- provider: 5
[ 67  78  80  88  99 107]
[4 3 5 0 1 2]
---- provider: 2
[ 67  78  80  88  99 107]
[5 1 3 4 0 2]
---- provider: 9
[ 67  78  80  88  99 107]
[0 5 1 3 2 4]
---- provider: 4
[ 67  78  80  88  99 107]
[2 4 5 1 0 3]
---- provider: 3
[ 67  78  80  88  99 107]
[0 5 1 4 3 2]


In [364]:
# II. Merge -------------------------------------------------------------


In [365]:
# assume the insurer_id in choices data 0,1,2,3,4,5 = 67  78  80  88  99 107, correspondingly 
dict_insurer_id = {0:67,
                   1:78,
                   2:80, 3:88, 4:99, 5:107}
choices = choices.replace({'insurer_id': dict_insurer_id})

In [366]:
# assume year in choices data and in providers data 0,1,2,3 = 2013, 2014, 2015, 2016, correspondingly 
providers['year'] = providers['year'] + 2013
choices['year'] = choices['year']+2013

In [371]:
# merge
df = pd.merge(choices,
         price_scales,
         how = 'inner',
         left_on = ['provider_id','insurer_id','year'],
         right_on = ['provider_id','insurer_id','year']).rename(columns = {'price':'price_scale'})
df = (pd.merge(df,
              providers,
              how = 'inner',
              left_on = ['provider_id','year'],
              right_on = ['provider_id','year'])
      .sort_values(ids)
      .reset_index(drop=True)
      .rename(columns = {'price':'price_provider_ave'}))
      

In [372]:
df.head()

,consumer_id,year,insurer_id,provider_id,age,female,num_dependents,price_scale,price_provider_ave,public
0,0,2015,78,13,43,1,1,0.919416,0.965927,1
1,1,2015,78,13,54,1,3,0.919416,0.965927,1
2,2,2013,67,7,60,1,1,0.740955,1.167918,0
3,3,2015,107,13,71,0,1,0.797114,0.965927,1
4,4,2015,107,8,46,1,0,1.290198,1.196602,0


In [373]:
# III. Generate Variables ------------------------------------------------

df['price'] = df['price_scale'] * df['price_provider_ave']

In [410]:
# IV. Summary Stats ------------------------------------------------------


In [405]:

# --------------------------------------
# Table 1
# --------------------------------------
table1 = (df.pivot_table(values=['age','price','consumer_id'], 
                    index='provider_id', 
                    aggfunc={'age':np.mean, 
                             'price':[np.mean,np.median],
                             'consumer_id': lambda x: len(x)/len(df)}  )
          .rename(columns = {'consumer_id': 'market share',
                             '<lambda>' : '%'} ) )
table1

age market share     price          
                  mean            %      mean    median
provider_id                                            
2            43.441129     0.037309  0.719325  0.770516
3            36.801641     0.026655  0.915381  0.893067
4            40.418196     0.076903  0.865461  0.881136
5            37.673884     0.088134  0.988896  0.982919
6            41.444573     0.068495  1.407686  1.402570
7            41.539748     0.116518  1.145333  1.173526
8            45.256625     0.028503  1.053652  1.126132
9            43.659064     0.016557  1.083929  0.941590
13           43.235136     0.540926  0.851757  0.908374

In [411]:
# --------------------------------------
# Table 2
# --------------------------------------
table2 = (df.pivot_table(values=['consumer_id','price', 'public'],
                        index = 'year',
                        aggfunc={'price':np.mean,
                                 'consumer_id': len,
                                 'public': lambda x: np.sum(x)/ len(x) } )
          .rename(columns = {'consumer_id': 'total number of cases',
                             'price': 'average price paid',
                             'public': 'share of public hospitals'})   )

table2

,total number of cases,average price paid,share of public hospitals
year,,,
2013,12108,0.855802,0.570862
2014,11867,0.830135,0.572259
2015,13528,1.004514,0.531195
2016,12807,1.067150,0.558913


In [ ]:
# --------------------------------------
# output to latex
# --------------------------------------

for tablename in ['table1', 'table2']:
    outputname = resultpath + '/' + tablename + '.tex'
    globals()[tablename].to_latex(outputname)

In [431]:
# V. Store data --------------------------------------------------------
print(len(df))
df.head()

50310


,consumer_id,year,insurer_id,provider_id,age,female,num_dependents,price_scale,price_provider_ave,public,price
0,0,2015,78,13,43,1,1,0.919416,0.965927,1,0.888089
1,1,2015,78,13,54,1,3,0.919416,0.965927,1,0.888089
2,2,2013,67,7,60,1,1,0.740955,1.167918,0,0.865375
3,3,2015,107,13,71,0,1,0.797114,0.965927,1,0.769954
4,4,2015,107,8,46,1,0,1.290198,1.196602,0,1.543853


In [432]:
filename = 'data_merged'
filekey = 'cases'
outputname = datapath + '/' + filename + '.h5'
df.to_hdf(outputname, key = filekey, complevel = 2, mode='w')